In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pyntcloud import PyntCloud
import torch.nn.functional as F

In [ ]:
class VoxelDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        return torch.from_numpy(sample).float()

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv3d(1, 32, kernel_size=4, stride=2, padding=1),  # Input: 1x64x64x64, Output: 32x32x32x32
            nn.ReLU(),
            nn.Conv3d(32, 64, kernel_size=4, stride=2, padding=1),  # Output: 64x16x16x16
            nn.ReLU(),
            nn.Conv3d(64, 128, kernel_size=4, stride=2, padding=1),  # Output: 128x8x8x8
            nn.ReLU()
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose3d(128, 64, kernel_size=4, stride=2, padding=1),  # Output: 64x16x16x16
            nn.ReLU(),
            nn.ConvTranspose3d(64, 32, kernel_size=4, stride=2, padding=1),  # Output: 32x32x32x32
            nn.ReLU(),
            nn.ConvTranspose3d(32, 1, kernel_size=4, stride=2, padding=1),  # Output: 1x64x64x64
            nn.Sigmoid()
        )
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
directory = "abc-dataset-ply/"
files = sorted([filename for filename in os.listdir(directory) if os.path.isfile(os.path.join(directory, filename))])
dataset = []

def convert_to_binvox(path):
    point_cloud = np.loadtxt(path, skiprows=12)[:, 0:3]
    df = pd.DataFrame(data=point_cloud, columns=['x','y','z'])
    cloud = PyntCloud(df)
    voxelgrid_id = cloud.add_structure("voxelgrid", n_x=64, n_y=64, n_z=64)
    voxelgrid = cloud.structures[voxelgrid_id]
    Binary_voxel_array = voxelgrid.get_feature_vector(mode="binary")
    dataset.append(Binary_voxel_array)

for i in files:
    path = os.path.join(directory, i)
    convert_to_binvox(path)

In [ ]:
train_data = dataset[:35]
test_data = dataset[35:]

# Create PyTorch datasets and dataloaders
train_dataset = VoxelDataset(train_data)
test_dataset = VoxelDataset(test_data)

# Determine the device (GPU or CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Initialize the model, optimizer, and loss function
autoencoder = Autoencoder().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=0.001)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)
print(train_loader)

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    for data in train_loader:
        optimizer.zero_grad()
        data = data.to(device)
        outputs = autoencoder(data.unsqueeze(1).to(device))
        loss = criterion(outputs, data.unsqueeze(1).to(device))
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

In [ ]:
autoencoder.eval()

# Create the "reconstructed_point_clouds" folder if it doesn't exist
output_folder = "reconstructed_point_clouds"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Use the trained autoencoder to reconstruct the test data
with torch.no_grad():
    for i, data in enumerate(test_loader):
        data = data.to(device)
        reconstructed_data = autoencoder(data.unsqueeze(1).to(device)).cpu().numpy().squeeze(1)

        # Convert the reconstructed binary voxel data to a 3D point cloud
        reconstructed_point_cloud = np.argwhere(reconstructed_data)  # Convert binary voxels to (x, y, z) points

        # Create a DataFrame with only x, y, and z columns
        df = pd.DataFrame(data=reconstructed_point_cloud)

        # Save the reconstructed point cloud to a PLY file
        output_path = os.path.join(output_folder, f"reconstructed_point_cloud_{i}.ply")
        df.to_csv(output_path, index=False, sep=' ', header=False)
        print(f"Reconstructed point cloud saved: {output_path}")